# Tutorial: Event Series Analysis 

Written as part of a PhD thesis in Physics by Jonathan F. Donges (donges@pik-potsdam.de) at the Potsdam Institute of Climate Impact Research (PIK) and Humboldt University Berlin,

Copyright 2008-2019.

Synchronisation measures of time series have been attracting attention in several research areas, including climatology or neuroscience.

Synchronisation can be understood as a measure of interdependence or strong correlation between time series. 
The main use cases of synchronisation are:
- Quantification of similarities in phase space between two time series
- Quantification of differences in phase space between two time series

A research example of synchronisation phenomena is the analysis electroencephalographic (EEG) signals as a major influencing factor to understand the communication within the brain. See [Quiroga et al., 2001](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.65.041903)

Two widely accepted measurement methods of synchronisation are **Event Synchronisation (ES)** and **Event Coincidence Analysis (ECA)**. The non-linear nature of these two methods makes them widely applicable for a wide range of utilizations. 
While ES does not include the difference in timescales when measuring synchrony, when using ECA a certain timescale has to be selected for analysis purposes.

For more background information consult [Odenweller et al., 2020](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.052213) or [Quiroga et al., 2001](https://www.researchgate.net/publication/11025829_Event_Synchronization_A_simple_and_fast_method_to_measure_synchronicity_and_time_delay_patterns).

### Event Synchronisation (ES)

As mentioned before, the parameter free method ES offers a fast and reliable method to measure synchronizations between time series.
The fundamental idea of the method is illustrated by the picture below (from [Odenweller et al., 2020](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.052213)):

![Event Synchronisation](images/EventSynchronisation.png)

Two events $l$ and $m$, from timeseries $i$ and $j$ are considered synchronous if they occur within a certain time interal $\tau$ retrieved from the data properties. The time interval $\tau$ is defined as:

$$\tau_{lm}^{ij}=\frac{1}{2}min\left[(t_{l+1}^{i}-t_{l}^{i}), \; (t_{l}^{i}-t_{l-1}^{i}), \; (t_{m+1}^{j}-t_{m}^{j}), \; (t_{m}^{j}-t_{m-1}^{j})\right]$$

From here the occurences of synchronised events in timeseries $i$ when given an event in $j$ gives:

$$c(i|j)=\sum_{l=2}^{s_i-1}\sum_{m=2}^{s_j-1}J_{lm}^{im}$$

whereby $J_{lm}^{im}$ counts the events that match the synchronization condition. For more detail on this, see [Odenweller et al., 2020](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.052213).

Finally, we can define the strength of event synchronisation between the timeseries $i$ and $j$ by:

$$Q_{ij}^{ES}=\frac{c(i|j)+c(j|i)}{\sqrt{(s_i-2)(s_j-2)}}$$

In the usual case for which the timeseries are not fully synchronised, $0 <= Q_{ij}^{ES} <= 1$. For total or absent synchronisation $Q_{ij}^{ES} = 1$ or $Q_{ij}^{ES} = 0$, respectively. 

To generate a network from a set of timeseries, we can consider the the  $Q_{ij}^{ES}$ values, as the coefficients of a square symmetric matrix $Q^{ES}$, from which an unidirected network from multivariate data can be constructed. 
It is to be noted that fully synchronised time series will adapt a value of $Q_{ii}^{ES} = Q_{jj}^{ES} = 1$. 

The advatage of ES is that no parameters, specially a delay specification of the two timeseries, has to selected a priori as the algorithm classifies two events as snychronized automatically. 

### Event coincidence analysis (ECA)

In contrary to ES, ECA is characterized by the incorporation of a *static (global) coincidence intervals*. Hereby a tolerance interval $\Delta T$ is defined so that for events in $j$ at $t_{m}^{i}$ preceding events in $i$ at $t_{l}^{i}$ obey to $0 \leq (t_{l}^{i} -  t_{m}^{j}) \leq $ $\Delta T$. The fundamental idea of the method is illustrated by the picture below (from [Odenweller et al., 2020](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.052213)):

![ECA](images/ECA.png)

It is to be noted, when determining the coincidence rate while performing ECA, the differentiation between *precursor* and *trigger* event coincidence rates applies. The former refers to the number of events in $i$ that precede the events in $j$. The trigger rates, on the other hand, quantify the events in $j$ that precede at least one event in $i$.  

Thus, the definition of the precursor event coincidence rate is defined as:

$$r_p(i|j;\Delta T,\tau) = \frac{1}{s_i-s_{i}^{'}}\sum_{l=1+s_{i}^{'}}^{s_i} \Theta \left[\sum_{m=1}^{s_j} 1_{[0,\Delta T]}\left[(t_{l}^{i}-\tau)-t_m^j\right]\right]$$

The trigger event coincidence rate on the other hand is defined as:

$$r_p(i|j;\Delta T,\tau)=\frac{1}{s_i-s_{i}^{''}}\sum_{m=1}^{s_j-s_j^{''}}\Theta\left[\sum_{l=1}^{s_i} 1_{[0,\Delta T]} \left[(t_l^i-\tau)-t_m^j\right]\right]$$

For detailed information on the calculation of e.g. $s_i^{''}$ or $s_j^{''}$, consult [Odenweller et al., 2020](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.101.052213). 

By changing the indices of the precursor and trigger rate, one gets the rate in the other directions e.g. for $r_t(i|j; \Delta T, \tau)$.

Subsequently the ECA analysis gives out four coincidence rates: the precursor and trigger rates evaluated in a bi-directional manner. 

To construct the event matrix composed by a single association measure, one can take the mean or the maximum value of the two tigger coincidence rates $r_p(i|j; \Delta T,\tau)$ and $r_p(j|i;\Delta;T,\tau)$ to compute the *degree of event synchrony* $Q_{ij}^{ECA, mean}$ or $Q_{ij}^{ECA, max}$.

## ES and ECA analysis of simple random event series

`pyunicorn` provides a class for event synchronization (ES) and
event coincidence analysis (ECA). In addition, a method for the generation of
binary event series from continuous time series data (method `make_event_matrix`, is included.

First we import all necessary packages

In [1]:
import numpy as np
from pyunicorn.eventseries import EventSeries

Next we create a toy data example.
The data structure has to follow an event matrix style, whereby the first axis represents the timesteps and the second axis covers the variables. Each variable at a specific timestep is either '1' if an event occured or '0' if it did not, e.g. for 3 variables with 10 timesteps the
eventmatrix could look like:

In [6]:
series = np.array([[0, 1, 0],
                   [1, 0, 1],
                   [0, 0, 0],
                   [1, 0, 1],
                   [0, 1, 0],
                   [0, 0, 0],
                   [1, 0, 0],
                   [0, 0, 1],
                   [0, 1, 0],
                   [0, 0, 0]])

Now we apply the class `EventSeries` to our data to initialize the process of synchronization analysis.

In [7]:
ev = EventSeries(series, taumax=1)
print(f"The initialized Event Series of our example is", ev)

The initialized Event Series of our example is EventSeries: 3 variables, 10 timesteps, taumax: 1.0, lag: 0.0


From here, the initialized data variables can be analyzed in terms of synchrony by applying the methods of Pyunicorn to calculate ES and ECA

Note: The argument `taumax` represents the maximum time difference between two events to be considered synchronous. Caution: For ES, using the default `taumax=np.inf` is sensible because of the intrinsic dynamic coincidence interval in ES. For ECA, `taumax` needs to be specified!

In [8]:
#Check for ES and ECA synchronization between two variables x and y of the generated dataset

print(ev.event_synchronization(series[:,0], series[:, 2]))
print(ev.event_coincidence_analysis(series[:,0], series[:, 2], taumax=1)) #Taumax has to be selected when calculating ECA; taumax=1 was selected for simplicity

(0.5, 0.5)
(0.5, 1.0, 1.0, 1.0)


- The return of the ES  method is [Event synchronization XY, Event synchronization YX]
- The return of the ECA methos is[ Precursor coincidence rate XY, Trigger coincidence rate XY, Precursor coincidence rate YX, Trigger coincidence rate YX]

If input data is not provided as an eventmatrix, the constructor tries to generate one from continuous time series data using the `make_event_matrix` method. Hereby the argument `threshold_method` needs to be specified along with the argument `threshold_values`. `threshold_method` can be 'quantile',  'value' or 1D numpy array  with entries 'quantile' or 'value' for each variable. If 'value' is selected one has to specify a number lying in the range of the array; for quantile a number between 0 and 1 has to be selected as it specifies the percentage of the array's values which should be included in the event matrix. Additionally one can specify by the argument `threshold_type` if the threshold should be applied 'above' or 'below' the specified `threshold_method`. A simple example of finding the synchrony between variables x and y of a dataset reflecting continuous time series, would be:

In [11]:
series = 10*np.random.rand(10,3)
series = series.astype(int)

#Initialize dataset as EventSeries
ev = EventSeries(series[:, :2], threshold_method = 'quantile', threshold_values = 0.5, threshold_types='below', taumax=1)

In [12]:
#Applying ES and ECA to find synchronity between variables x and y
print(ev.event_synchronization(series[:,0], series[:, 1]))
print(ev.event_coincidence_analysis(series[:,0], series[:, 1], taumax=1))

(0.42857142857142855, 0.42857142857142855)
(1.0, 1.0, 1.0, 1.0)


In [13]:
#When one just wants to get the precursor event coincidence rates; _eca_coincidence_rate method can be used
ev._eca_coincidence_rate(series[:, 0], series[:, 1], window_type = 'advanced')

(1.0, 1.0)

To get the matrix to construct the functional unidirectional network described in the Introduction to ES and ECA at the beginning of the tutorial, the method `event_series_analysis` can be used. Hereby the method to anylize synchrony has to be specified by the argument `method` as *ES* or *ECA*.

For detailed information on the calculation of the matrix and the required arguments when selecting *ES* or *ECA*, consult [pyunicorn Github](https://github.com/pik-copan/pyunicorn/blob/master/src/pyunicorn/eventseries/event_series.py)/from line 704.

The return is a NxN matrix of the chosen event series where N is the number of variables. 

In [14]:
matrix_ES = ev.event_series_analysis(method='ES')
print(f"Matrix of Es strenghts is", matrix_ES)

matrix_ECA = ev.event_series_analysis(method='ECA', symmetrization = 'mean', window_type = 'advanced')
print(f"Matrix of ECA strengths is", matrix_ECA) #Symmetrization was selected according to Odenweller et al., 2020

Matrix of Es strenghts is [[0.         0.16666667]
 [0.16666667 0.        ]]
Matrix of ECA strengths is [[0.    0.875]
 [0.875 0.   ]]


Approaches of complex network theory can be now applied to the matrices calculated, as part of non-linear timeseries analysis. See jupyter notebook `tutorial_recurrenceNetwork`.

### Significance level calculations

The signifcance levels of event synchronisation can be calculated using pyunicorn too. The package provides to methods `_empirical_percentiles` and `event_analysis_significance`, to either calcuate the p-values through a Monte Carlo approach or the signifcance levels ( 1 - p-values) through a Poisson process from an event matrix, repectively.

For detailed information on the calculation of the significance levels consult [pyunicorn Github](https://github.com/pik-copan/pyunicorn/blob/master/src/pyunicorn/eventseries/event_series.py)/from line 830.

In [15]:
MonteCarlo_ES = ev._empirical_percentiles(method='ES', n_surr=1000)
print(MonteCarlo_ES)
MonteCarlo_ECA = ev._empirical_percentiles(method='ECA', n_surr=1000, symmetrization = 'mean', window_type = 'advanced')
print(MonteCarlo_ECA)

[[0.    0.119]
 [0.123 0.   ]]
[[0.    0.609]
 [0.609 0.   ]]


In [16]:
Poisson_ES = ev.event_analysis_significance(method='ES', n_surr=1000)
print(Poisson_ES)
Poisson_ECA = ev.event_analysis_significance(method='ECA', n_surr=1000, symmetrization = 'mean', window_type = 'advanced')
print(Poisson_ECA)

[[0.    0.12 ]
 [0.112 0.   ]]
[[0.    0.609]
 [0.609 0.   ]]


##  ES and ECA analysis to generate a climate network

Further application of the ES and ECA analysis is the possible generation of a climate network using the resulting ES and ECA matrices from the calculations shown above. This can be done by using the pyunicorn Class `EventSeriesClimateNetwork`. An example will be shown next. 

Note: If other applications of event series networks are desired, use the `EventSeries` Class together with `Network` Class

In [17]:
#import additonal packages to generate Climate Networks 
from pyunicorn.climate.eventseries_climatenetwork import EventSeriesClimateNetwork
from pyunicorn.core import Data #Encapsulates general spatio-temporal data

climate: Package Ngl could not be loaded. Some functionality in class MapPlots might not be available!


We shall use the small test climate dataset provided by the Class  `EventSeriesClimateNetwork`. See [pyunicorn Github repository describing Class EventSeriesClimateNetwork](https://github.com/pik-copan/pyunicorn/blob/master/src/pyunicorn/climate/eventseries_climatenetwork.py) for more specifications.

In [18]:
#Generate Test dataset and returns ClimateData instance for testing purposes
data = Data.SmallTestData() #Return test data set of 6 time series with 10 sampling points each
print(f"Information of dataset generated", data)

#To actually print out the "observables" of the toy climate dataset
print(f"Observables of dataset generated", data.observable())

Information of dataset generated Data: 6 grid points, 60 measurements.
Geographical boundaries:
         time     lat     lon
   min    0.0    0.00    2.50
   max    9.0   25.00   15.00
Observables of dataset generated [[ 0.00000000e+00  1.00000000e+00  1.22464680e-16 -1.00000000e+00
  -2.44929360e-16  1.00000000e+00]
 [ 3.09016994e-01  9.51056516e-01 -3.09016994e-01 -9.51056516e-01
   3.09016994e-01  9.51056516e-01]
 [ 5.87785252e-01  8.09016994e-01 -5.87785252e-01 -8.09016994e-01
   5.87785252e-01  8.09016994e-01]
 [ 8.09016994e-01  5.87785252e-01 -8.09016994e-01 -5.87785252e-01
   8.09016994e-01  5.87785252e-01]
 [ 9.51056516e-01  3.09016994e-01 -9.51056516e-01 -3.09016994e-01
   9.51056516e-01  3.09016994e-01]
 [ 1.00000000e+00  1.22464680e-16 -1.00000000e+00 -2.44929360e-16
   1.00000000e+00  3.67394040e-16]
 [ 9.51056516e-01 -3.09016994e-01 -9.51056516e-01  3.09016994e-01
   9.51056516e-01 -3.09016994e-01]
 [ 8.09016994e-01 -5.87785252e-01 -8.09016994e-01  5.87785252e-01
   8.090

The Class `ClimateData` is applied by definition on the Test dataset for generating and analyzing complex climate networks. See [Github repository describing Class ClimateData](https://github.com/pik-copan/pyunicorn/blob/1e0eb4d8e6e4ec07254d425975b154ee1519dfe1/src/pyunicorn/climate/climate_data.py#L36). \
**Note**: `ClimateData` Class shall be applied to a newly generated dataset of observables if not `SmallTestData()`

In [19]:
#Apply the EventSeriesClimateNetwork to the toy climate data generated
climate_ES = EventSeriesClimateNetwork(data, taumax=16.0,
      threshold_method='quantile', threshold_values=0.8,
        threshold_types='above') #'ES' method is default

print(f"The output of event series analysis is", climate_ES)

Extracting network adjacency matrix by thresholding...
Setting area weights according to type surface ...
Setting area weights according to type surface ...
The output of event series analysis is EventSeriesClimateNetwork:
EventSeries: 6 variables, 10 timesteps, taumax: 16.0, lag: 0.0
ClimateNetwork:
GeoNetwork:
SpatialNetwork:
Network: directed, 6 nodes, 0 links, link density 0.000.
Geographical boundaries:
         time     lat     lon
   min    0.0    0.00    2.50
   max    9.0   25.00   15.00
Threshold: 0
Local connections filtered out: False
Type of event series measure to construct the network: directedES


In [20]:
#Try out for ECA analysis
climate_ECA = EventSeriesClimateNetwork(data, method = 'ECA', taumax=16.0,
      threshold_method='quantile', threshold_values=0.8,
        threshold_types='above') #'ES' method is default

print(f"The output of event series analysis is", climate_ECA)

Extracting network adjacency matrix by thresholding...
Setting area weights according to type surface ...
Setting area weights according to type surface ...
The output of event series analysis is EventSeriesClimateNetwork:
EventSeries: 6 variables, 10 timesteps, taumax: 16.0, lag: 0.0
ClimateNetwork:
GeoNetwork:
SpatialNetwork:
Network: directed, 6 nodes, 0 links, link density 0.000.
Geographical boundaries:
         time     lat     lon
   min    0.0    0.00    2.50
   max    9.0   25.00   15.00
Threshold: 0
Local connections filtered out: False
Type of event series measure to construct the network: directedECA
